In [5]:
from gensim.models import Word2Vec, KeyedVectors

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install bertopic[visualization]

     |████████████████████████████████| 57 kB 2.6 MB/s 
     |████████████████████████████████| 78 kB 5.9 MB/s 
     |████████████████████████████████| 86 kB 6.2 MB/s 
     |████████████████████████████████| 26.5 MB 1.2 MB/s 
     |████████████████████████████████| 15.7 MB 22.8 MB/s 
     |████████████████████████████████| 6.4 MB 31.6 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 3.4 MB 36.7 MB/s 
     |████████████████████████████████| 6.8 MB 22.8 MB/s 
     |████████████████████████████████| 1.2 MB 66.0 MB/s 
     |████████████████████████████████| 61 kB 449 kB/s 
     |████████████████████████████████| 636 kB 55.2 MB/s 
     |████████████████████████████████| 895 kB 56.7 MB/s 
     |████████████████████████████████| 3.3 MB 40.9 MB/s 
     |████████████████████████████████| 1.1 MB 62.6 MB/s 
  Created wheel for hdbscan: filename=hdbscan-0.8.27-cp37-cp37m-linux

In [1]:
# Colab może mieć problemy z czytaniem HDFów. Konieczny jest upgrade tables
!pip install --upgrade tables

     |████████████████████████████████| 5.9 MB 3.8 MB/s 
  Attempting uninstall: tables
    Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4


In [7]:
import pandas as pd
import numpy as np
from bertopic import BERTopic
import gensim
from gensim.models import FastText, KeyedVectors
import hdbscan
import umap

In [8]:
# Ścieżka do katalogu z notebookami na Colabie
path = '../content/drive/MyDrive/Colab Notebooks/'

In [9]:
df = pd.read_hdf(path+'outputs/budzety_clean.hd5', key='df_all')

In [10]:
df = df.reset_index()
del df['index']
df

,rok,miasto,nazwa,WSZYSTKO,tokens,zbiornik,sadzić,estetyczny,lęgowy,ekologia,rekultywacja,trawnik,nasadzić,bylina,łąka,park,ogrodniczy,ogród,ozdobny,roślina,kwietny,ekologiczny,zazielenić,parkowy,leśny,natura,las,kwiatowy,roslin,zasadzić,gaj,kwiat,krzew,drzewo,trawiasty,ptasi,sztuczny,zielony,przyroda,przyrodniczy,żywopłot,dębowy,kasztanowiec,leś,powietrze,dziki,naturalny,trawniki,antysmogowy,estetyka,kwietnik,deszczowy,kwitnąć,ogrodowy,pielęgnacja,smog,zieleniec,names,szkola,ulica,oswietlenie
0,2020,GDYNIA,Projektistatus,0,[projektistatus],False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,projektistatus,False,False,False
1,2020,GDYNIA,Doposażenie parku pomiędzy Estakadą Kwiatkowsk...,3,"[doposażenie, park, pomiędzy, estakada, kwiatk...",False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,doposażenie park pomiędzy estakada kwiatkowski...,False,True,False
2,2020,GDYNIA,Cykliczne warsztaty dla seniorów z dzielnicy O...,0,"[cykliczny, warsztat, senior, dzielnica, obłuż...",False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,cykliczny warsztat senior dzielnica obłuże roz...,False,False,False
3,2020,GDYNIA,"Ogród Sąsiedzki przy Dzikim Sadzie, ul. Źródlana",3,"[ogród, sąsiedzki, dziki, sadź, ul, źródlana]",False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,ogród sąsiedzki dziki sadź ul źródlana,False,True,False
4,2020,GDYNIA,Montaż słupów oświetleniowych na chodniku przy...,0,"[montaż, słup, oświetleniowy, chodnik, ul, nis...",False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,montaż słup oświetleniowy chodnik ul niski odc...,False,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23025,2020,KRAKÓW,Zielony skwer na Zabtociu,1,"[zielony, skwer, zabtociu]",False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,zielony skwer zabtociu,False,False,False
23026,2020,KRAKÓW,Znani Biatopradniczanie dla Mieszkanców,0,"[znany, mieszkanców]",False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,znany mieszkanców,False,False,False
23027,2020,KRAKÓW,ZRÓBMY SKWER 

In [11]:
docs = list(df.loc[:, 'names'].values)

docs = [str(elem) for elem in docs] 

In [12]:
model = KeyedVectors.load_word2vec_format(path+'models/glove_ekolo.bin', binary=True)

In [23]:
# https://towardsdatascience.com/fse-2b1ffa791cf9
# Smooth inverse frequency - rodzaj embeddingu zdań, który bierze pod uwagę
# częstotliwość występowania słów. Dla każdego słowa liczona jest waga
# ze wzoru a/(a+p(w)) gdzie a to hiperparametr, a p(w) to częstotliwość występowania
# słowa w zdaniu.
# Słowa występujące rzadziej mają większe wagi.

def sif_embeddings(sentences, model, alpha=1e-3):
    """Compute the SIF embeddings for a list of sentences
    Parameters
    ----------
    sentences : list
        The sentences to compute the embeddings for
    model : `~gensim.models.base_any2vec.BaseAny2VecModel`
        A gensim model that contains the word vectors and the vocabulary
    alpha : float, optional
        Parameter which is used to weigh each individual word based on its probability p(w).
    Returns
    -------
    numpy.ndarray 
        SIF sentence embedding matrix of dim len(sentences) * dimension
    """
    
    vlookup = model.wv.vocab  # Gives us access to word index and count
    vectors = model.wv        # Gives us access to word vectors
    size = model.vector_size  # Embedding size
    
    Z = 0
    for k in vlookup:
        Z += vlookup[k].count # Compute the normalization constant Z
    
    output = []
    
    # Iterate all sentences
    for s in sentences:
        count = 0
        v = np.zeros(size, dtype=np.float32) # Summary vector
        # Iterare all words
        for w in s:
            # A word must be present in the vocabulary
            if w in vlookup:
                for i in range(size):
                    v[i] += ( alpha / (alpha + (vlookup[w].count / Z))) * vectors[w][i]
                count += 1 
                
        if count > 0:
            for i in range(size):
                v[i] *= 1/count
        output.append(v)
    return np.vstack(output).astype(np.float32)

In [29]:
X = sif_embeddings(docs, model)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


In [30]:
# umap_model = umap.UMAP(n_neighbors=15,
#                        n_components=10,
#                        min_dist=0.0,
#                        metric='cosine',
#                        low_memory=False)

# # HDBSAN - klasteryzacja dokumentów
# hdbscan_model = hdbscan.HDBSCAN(min_cluster_size=10,
#                                 min_samples=1,
#                                 metric='euclidean',
#                                 cluster_selection_method='eom',
#                                 prediction_data=True)

model_bert = BERTopic(top_n_words=30,
                       calculate_probabilities=True,
                       verbose=True,
                       language='polish')

topics, probs = model_bert.fit_transform(docs, X)


/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)
2021-12-31 16:19:27,991 - BERTopic - Reduced dimensionality with UMAP
2021-12-31 19:15:26,813 - BERTopic - Clustered UMAP embeddings with HDBSCAN


In [31]:
model_bert

In [32]:
# !!!!!! WAŻNE !!!!!!
#
# Jeżeli masz w  folderze models pliki:
# 'main_bert_topic_fasttext.model' i 'main_bert__fasttext_topics_probs.npy' to nie uruchamiaj
# tej komórki 

model_bert.save(path+'models/main_bert_topic_glove_from_global.model')
with open(path+'models/main_bert_glove_from_global_topics_probs.npy', 'wb') as f:
    np.save(f, topics)
    np.save(f, probs)

/usr/local/lib/python3.7/dist-packages/scipy/sparse/_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [ ]:
# Wczytanie zapisanych modeli
model_bert = BERTopic.load(path+'models/main_bert_topic_glove_from_global.model')
with open(path+'models/main_bert_glove_from_global_topics_probs.npy', 'rb') as f:
    topics = np.load(f)
    probs = np.load(f)

In [15]:
model_bert.get_representative_docs()

{0: ['zakup książka audiobook użytkownik biblioteka miejski filia',
  'zakup książka nowość wydawniczy czytelnik filia',
  'zakup nowosci wydawniczy czasopismo dofinansowanie impreza kulturalny dorostych dziecko mtodziezy filia miejski biblioteka publiczny katowicach bogucicach'],
 1: ['', '', ''],
 2: ['modernizacja boisko sportowy sp',
  'budowa boisko siatkówka plażowy sp',
  'budowa boisko piłka nożny dębowy polano'],
 3: ['orkiestra promenadowy koncert wakacyjny',
  'koncert wiosna muzyk sacrum',
  'koncert kolędowy zerzeniu'],
 4: ['nauka rucho bezpłatny zajęcie terapia integracja sensoryczny dziecko',
  'zajęcie terapia integracja sensoryczny sp',
  'musicon nowatorski wsparcie dziecko terapia psychologiczno pedagogiczny'],
 5: ['nasadzenia ul cienistej',
  'dokonczenie sciezki utwardzenie pieszy spinajacej ul pultuska niborska przyjazni ul grunwaldzka',
  'skatepark ul malinowy'],
 6: ['składać robota zero złoty warsztat robotyka dziecko wykorzystanie klocki lego urządzenie mob

In [33]:
# Sprawdzenie tematów
model_bert.get_topics()

{-1: [('drzewo', 0.0009638419090057918),
  ('park', 0.0009501495333781422),
  ('zielony', 0.000910349054501957),
  ('zabawa', 0.0009023005596751393),
  ('siq', 0.0008991775927293489),
  ('etap', 0.0008943201151762408),
  ('blok', 0.0008846432250631042),
  ('teren', 0.0008763500838566103),
  ('plac', 0.0008706930820523107),
  ('pas', 0.0008670629017523015),
  ('nowy', 0.0008619760650170528),
  ('boisko', 0.0008607330686311135),
  ('sp', 0.0008598353202797444),
  ('remonta', 0.0008538417742524824),
  ('budowa', 0.0008493017959044304),
  ('rowerowy', 0.0008466391838121333),
  ('ul', 0.0008441325435928134),
  ('wykonanie', 0.0008432979099615043),
  ('aleja', 0.0008428613845171884),
  ('dzielnica', 0.0008419217513142332),
  ('parking', 0.0008397462741848477),
  ('zieleń', 0.0008319436015294856),
  ('polski', 0.0008298960217781636),
  ('ulica', 0.0008289653737261486),
  ('mieszkaniec', 0.0008268552600546767),
  ('relaks', 0.0008224942271954165),
  ('biblioteka', 0.0008216489607446541),
  ('s

In [34]:
# W domyślnym modelu mamy 800 tematów
# ograniczmy ich ilość
N_TOPICS = 20

In [35]:
# Redukcja tematów 
new_topics, new_probs = model_bert.reduce_topics(docs, topics, probabilities=probs,
 nr_topics=N_TOPICS)

# Zapiszmy nowe tematy
# with open(path+'models/new_bert_topics_probs.npy', 'wb') as f:
#     np.save(f, new_topics)
#     np.save(f, new_probs)

2021-12-31 19:40:23,623 - BERTopic - Reduced number of topics from 1106 to 21


In [36]:
# Podgląd tematów
model_bert.get_topic_info() # Bardzo duża liczba nieprzypisanych tematów

,Topic,Count,Name
0,-1,20653,-1_ul_plac_zabawa_budowa
1,0,174,0_chodnik_ul_angielski_słownictwo
2,1,163,1_saneczkowy_budowa_ul_górka
3,2,159,2_ul_osiedle_nozna_stołówka
4,3,145,3_miejsce_workout_street_górczyna
5,4,143,4_okoto_zielony_park_trasa
6,5,139,5_ul_energia_dziesięć_chodnik
7,6,136,6_ekologicznie_komar_ul_przeciw
8,7,132,7_chcieć_zabawa_sacrum_autokarowy
9,8,127,8_nietoperz_wiewiórka_zawieszenie_ptak


In [ ]:
topic_dict = dict()
for n in range(-1, N_TOPICS):
  topic_dict[n] = [name for name, prob in model_bert.get_topic(n)]
  print(f'{n} {topic_dict[n]}', sep='\n')

-1 ['ul', 'plac', 'budowa', 'zabawa', 'ulica', 'plac zabawa', 'park', 'dziecko', 'zielony', 'chodnik', 'teren', 'remonta', 'miejsce', 'osiedle', 'rowerowy', 'droga', 'modernizacja', 'skwer', 'mieszkaniec', 'wraz']
0 ['szkoła', 'boisko', 'podstawowy', 'szkoła podstawowy', 'sp', 'sportowy', 'zespół', 'budowa boisko', 'budowa', 'modernizacja', 'zespół szkoła', 'szkolny', 'boisko sportowy', 'teren', 'lo', 'modernizacja boisko', 'ul', 'plac', 'zabawa szkoła', 'kompleks']
1 ['zajęcie', 'dziecko', 'młodzież', 'dziecko młodzież', 'warsztat', 'samoobrona', 'kobieta', 'ruchowy', 'sportowy', 'kurs', 'samoobrona kobieta', 'dorosły', 'bezpłatny', 'bezpłatny zajęcie', 'senior', 'elementy samoobrona', 'zajęcie fizyczny', 'poprawiać koordynacja', 'koordynacja ruchowy', 'fizyczny elementy']
2 ['pies', 'wybieg', 'wybieg pies', 'park', 'zwierzę', 'psi', 'bezdomność zwierzę', 'bezdomność', 'zwierzę teren', 'teren osiedle', 'kota', 'kot', 'żyć', 'ograniczanie', 'ograniczanie bezdomność', 'psi park', 'wolno

In [ ]:
# Dalsze kroki są takie same jak w poprzednich notatnikach.